# TGIS 501
## The Final Project!
### Ben Clark

This project will take an uploaded kml file containing data collected from the field and convert it into individual shapefiles, clean and reformat the attributes, and run certain geoprocesses to minimalize the work to be perfomed in ArcMap. The goal is to make it completely automated.

#### Pseudocode

 - Scan a designated folder for files
 - If new files, run certain geoprocessing tools on each file
 - Geoprocessing tools: KML to shapefile conversion, Export to individual shapefiles, clean and organize attributes, Buffer specific shapefiles, merge buffers, and clip to harvest unit boundary
 - Output will be multiple shapefiles saved in specific job folder
 - Methods: check for file, convert to shapefile, export individual, add attributes, buffer, merge, clip
 

In [ ]:
import os
from os import listdir
from os.path import isfile, join
import arcpy
import datetime
import re

#Define paths to different folders
input_path = "C:\Users\clarkb7\Documents\Test\Upload"
output_path = "C:\Users\clarkb7\Documents\Test"
workspace = ""
jobID = ""
og_file = ""
arcpy.env.overwriteOutput = True
arcpy.env.workspace = workspace

#Checks for the existence of any files in a given path NOTE: Empty directoris within the path will return FALSE (Files only!)
# IN: A path to check
# OUT: True if files, False if no files
def checkForNewFiles(path): 
    return any(isfile(join(path, i)) for i in listdir(path))

def ensure_dir(path):
    global workspace
    if not os.path.isdir(path+"\\"+jobID):
        os.makedirs(path+"\\"+jobID)
        workspace = path+"\\"+jobID
        print("Output folder created")
        return workspace
    else:
        workspace = path+"\\"+jobID
        print("Output folder already exists")
        
def getFileName(files):
    global jobID
    global og_file
    og_file = re.split("[.]", files)[0]
    jobID = re.split(" ", og_file)[0]
    return jobID, og_file

def convertKML(fileName):
    kmlLayer = arcpy.conversion.KMLToLayer(fileName, workspace)
    arcpy.management.CopyFeatures(jobID+"\Points", "Points.shp")
    arcpy.management.CopyFeatures(jobID+"\Polylines", "Lines.shp")
    arcpy.management.CopyFeatures(jobID+"\Polygons", "Polygons.shp")
    arcpy.management.Delete(kmlLayer)
    arcpy.management.Delete(og_file+".gdb")

def makeShapefiles(fileName):

    if fileName == "Points.shp":
        arcpy.management.MakeFeatureLayer(fileName, fileName+"_lyr")
        
        arcpy.management.SelectLayerByAttribute(fileName+"_lyr", "NEW_SELECTION", '"Name" = \'PIP\'')
        if int(arcpy.management.GetCount(fileName+"_lyr").getOutput(0) > 0):
            arcpy.management.CopyFeatures(fileName+"_lyr", "PIP.shp")
            
        arcpy.management.SelectLayerByAttribute(fileName+"_lyr", "NEW_SELECTION", '"Name" = \'NP CONF\'')
        if int(arcpy.management.GetCount(fileName+"_lyr").getOutput(0) > 0):
            arcpy.management.CopyFeatures(fileName+"_lyr", "NP_CONF.shp")
        
    elif fileName == "Lines.shp":
        arcpy.management.MakeFeatureLayer(fileName, fileName+"_lyr")
        
        arcpy.management.SelectLayerByAttribute(fileName+"_lyr", "NEW_SELECTION", '"Name" = \'NP\'')
        if int(arcpy.management.GetCount(fileName+"_lyr").getOutput(0) > 0):
            arcpy.management.CopyFeatures(fileName+"_lyr", "NP.shp")
        
        arcpy.management.SelectLayerByAttribute(fileName+"_lyr", "NEW_SELECTION", '"Name" = \'F\'')
        if int(arcpy.management.GetCount(fileName+"_lyr").getOutput(0) > 0):
            arcpy.management.CopyFeatures(fileName+"_lyr", "F.shp")
            
        arcpy.management.MakeFeatureLayer("NP.shp", "NP_lyr")
        arcpy.management.SelectLayerByAttribute("NP_lyr", "NEW_SELECTION", '"Buff_Dist" > 0')
        arcpy.management.CopyFeatures("NP_lyr", "NP_Buff.shp")
    
        arcpy.management.MakeFeatureLayer("F.shp", "F_lyr")
        arcpy.management.SelectLayerByAttribute("F_lyr", "NEW_SELECTION", '"Buff_Dist" > 0')
        arcpy.management.CopyFeatures("F_lyr", "F_Buff.shp")
            
    elif fileName == "Polygons.shp":
        arcpy.management.MakeFeatureLayer(fileName, fileName+"_lyr")
        
        arcpy.management.SelectLayerByAttribute(fileName+"_lyr", "NEW_SELECTION", '"Name" = \'Unit_Boundary\'')
        if int(arcpy.management.GetCount(fileName+"_lyr").getOutput(0) > 0):
            arcpy.management.CopyFeatures(fileName+"_lyr", "Unit_Boundary.shp")

def cleanAttributes(fileName):
    arcpy.management.AddField(fileName+".shp", "Date", "TEXT")
    arcpy.management.AddField(fileName+".shp", "JobID", "TEXT")
    arcpy.management.MakeFeatureLayer(fileName, fileName + "_lyr")
    if int(arcpy.management.GetCount(fileName + "_lyr").getOutput(0) > 0):
        cursor = arcpy.UpdateCursor(fileName + "_lyr")
        for row in cursor:
            row.setValue("Date", str(datetime.date.today()))
            row.setValue("JobID", jobID)
            cursor.updateRow(row)
        del cursor, row
    arcpy.management.DeleteField(fileName, 
                             ["FolderPath", "SymbolID", "AltMode", "Base", "TimeSpan", "TimeStamp", "EndTime", "Snippet", "PopupInfo", "LabelID", "HasLabel"])

if checkForNewFiles(input_path) == True:
    for files in os.listdir(input_path):
        getFileName(files)
        ensure_dir(output_path)
        convertKML(files)
        #makeShapefiles(files)
        #cleanAttributes(files)
        #os.remove(files)
        
if checkForNewFiles(workspace) == True:
    arcpy.management.CreateFeatureclass(workspace, "RMZ_Merge.shp", "POLYGON", "DISABLED", "DISABLED", "DISABLED", arcpy.Describe(path+"\\"+jobID+".shp").spatialReference)
    for files in os.listdir(workspace):
        if files != jobID + ".shp":
            name = getFileName(files)
            if getFileName(files) == "NP" or getFileName(files) == "F":
                arcpy.analysis.Buffer(files, getFileName(files) + "_buff.shp", "Buffer_Distance", "FULL", "FLAT")
            if getFileName(files) == "PIP":
                arcpy.analysis.Buffer(files, getFileName(files) + "_buff.shp", "56 Feet")
            arcpy.management.Merge(files, "RMZ_Merge.shp")

        arcpy.analysis.Clip("RMZ_Merge.shp", "Unit_Boundary.shp", "RMZ.shp")

